# Pacific Northwest Cities Analysis

In [2]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [7]:
# Web scraping to extract preliminary data
# List of cities in Washington State
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Washington", header=[0])
df_wa = tables[0]
df_wa = df_wa.iloc[1:,0:1]
df_wa.rename(columns={'Name': 'City'}, inplace=True)
df_wa = df_wa.reset_index(drop=True)
df_wa['State']='WA'
df_wa['Latitude']=''
df_wa['Longitude']=''
df_wa.head()

,City,State,Latitude,Longitude
0,Aberdeen,WA,,
1,Airway Heights,WA,,
2,Algona,WA,,
3,Anacortes,WA,,
4,Arlington,WA,,


In [9]:
# Web scraping to extract preliminary data
# List of cities in Oregon State
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_in_Oregon")
df_or = tables[1]
df_or = df_or.iloc[:,1:2]
df_or['City'] = df_or['City'].str.replace('†', '')
df_or = df_or.reset_index(drop=True)
df_or['State']='OR'
df_or['Latitude']=''
df_or['Longitude']=''
df_or.head()

,City,State,Latitude,Longitude
0,Portland,OR,,
1,Salem,OR,,
2,Eugene,OR,,
3,Gresham,OR,,
4,Hillsboro,OR,,


In [10]:
# Web scraping to extract preliminary data
# List of cities in Idaoho State
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_in_Idaho")
df_id = tables[0]
df_id = df_id.iloc[:,1:2]
df_id['City'] = df_id['City'].str.replace('†', '')
df_id = df_id.reset_index(drop=True)
df_id['State']='ID'
df_id['Latitude']=''
df_id['Longitude']=''
df_id.head()

,City,State,Latitude,Longitude
0,Boise,ID,,
1,Meridian,ID,,
2,Nampa,ID,,
3,Idaho Falls,ID,,
4,Pocatello,ID,,


In [11]:
# Consolidate web scraping results
cities = [df_wa, df_or, df_id]
df_pnw = pd.concat(cities)
df_pnw = df_pnw.reset_index(drop=True)
df_pnw.shape

(653, 4)

In [12]:
# Add Mapquest credentials to look up coordinates of each city
MAPQUEST_KEY = 'urHQ4hSPeGCSSj2BuOGZb5Lo421AeAc5'
MAPQUEST_SECRET = 'Np3xeXgW2GAdLBoj'

In [13]:
for index, row in df_pnw.iterrows():
    location = row['City'] + "," + row['State']
    url = 'https://www.mapquestapi.com/geocoding/v1/address?key={}&inFormat=kvp&outFormat=json&location={}&thumbMaps=false'.format(
    MAPQUEST_KEY, 
    location)
    response = requests.get(url).json()
    df_pnw.at[index,'Latitude'] = response['results'][0]['locations'][0]['latLng']['lat']
    df_pnw.at[index,'Longitude'] = response['results'][0]['locations'][0]['latLng']['lng']
    
df_pnw.to_csv('PNWCities.csv', sep=',',index=False)

In [14]:
df_pnw.head()

,City,State,Latitude,Longitude
0,Aberdeen,WA,46.9755,-123.816
1,Airway Heights,WA,47.643,-117.593
2,Algona,WA,47.2791,-122.25
3,Anacortes,WA,48.5054,-122.632
4,Arlington,WA,48.1913,-122.126


In [15]:
# Use geopy library to get the latitude and longtitude values of WA
address = 'Washington State, USA'

geolocator = Nominatim(user_agent="northwest_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington are 47.2868352, -120.2126139.


In [16]:
# Map the PNW with coordinates info
map_pnw = folium.Map(location=[latitude, longitude], zoom_start=5)

# Add markers
for lat, lng, name in zip(df_pnw['Latitude'], df_pnw['Longitude'], df_pnw['City']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pnw)  
    
map_pnw

In [17]:
# Add Foursquare credentials to look up top venues of each city
CLIENT_ID = 'WFSWOS4SU2FHJISFOLFWZQJUAFAJJJ324KKGA24CWMJZCTRT' # your Foursquare ID
CLIENT_SECRET = 'ETX1WAMFIETM4RCROHGDLQQL3V4NSVIIEWYZ5NSX0YGCRDRI' # your Foursquare Secret
VERSION = '20200606'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WFSWOS4SU2FHJISFOLFWZQJUAFAJJJ324KKGA24CWMJZCTRT
CLIENT_SECRET:ETX1WAMFIETM4RCROHGDLQQL3V4NSVIIEWYZ5NSX0YGCRDRI


In [20]:
# Get geo info of the first city in the dataframe
city_latitude = df_pnw.loc[0, 'Latitude'] # City latitude value
city_longitude = df_pnw.loc[0, 'Longitude'] # City longitude value

city_name = df_pnw.loc[0, 'City'] # Name
city_state = df_pnw.loc[0, 'State'] # State

print('Latitude and longitude values of {}, {} are: {}, {}.'.format(city_name,
                                                               city_state,
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Aberdeen, WA are: 46.97551, -123.815517.


In [21]:
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9787a04c1f6729025af446'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 10000,
  'headerLocation': 'Aberdeen',
  'headerFullLocation': 'Aberdeen',
  'headerLocationGranularity': 'city',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 46.98616500184368,
    'lng': -123.7037284668247},
   'sw': {'lat': 46.9434858956073, 'lng': -123.91647086388492}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d828e4b5e70224bbc0ce608',
       'name': 'Oceana Spa',
       'location': {'address': '501 W Wishkah St',
        'lat': 46.971636655386625,
        'lng': -123.82257641754265,
        'labeledLatLngs

In [29]:
# Create a function to extract the category of venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
# Clean the data and structure it as a dataframe
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
venues_filtered = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
venues_filtered['categories'] = venues_filtered.apply(get_category_type, axis=1)

# clean columns by keeping only the last term
venues_filtered.columns = [col.split(".")[-1] for col in venues_filtered.columns]

venues_filtered.head()

,name,categories,lat,lng
0,Oceana Spa,Spa,46.971637,-123.822576
1,Rediviva Restaurant,New American Restaurant,46.974826,-123.817105
2,Sucher & Sons Star Wars Shop,Toy / Game Store,46.976254,-123.813901
3,Amore' Italian Restaurant,Italian Restaurant,46.973007,-123.817937
4,Billy's Bar and Grill,Bar,46.975505,-123.813540


In [31]:
# Create a function to repeat the same process to all cities
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# Run the function on each city and store in new dataframe
pnw_venues = getNearbyVenues(names=df_pnw['City'],
                                   latitudes=df_pnw['Latitude'],
                                   longitudes=df_pnw['Longitude'])

Aberdeen
Airway Heights
Algona
Anacortes
Arlington
Asotin
Auburn
Bainbridge Island
Battle Ground
Bellevue
Bellingham
Benton City
Bingen
Black Diamond
Blaine
Bonney Lake
Bothell
Bremerton
Brewster
Bridgeport
Brier
Buckley
Burien
Burlington
Camas
Carnation
Cashmere
Castle Rock
Centralia
Chehalis
Chelan
Cheney
Chewelah
Clarkston
Cle Elum
Clyde Hill
Colfax
College Place
Colville
Connell
Cosmopolis
Covington
Davenport
Dayton
Deer Park
Des Moines
DuPont
Duvall
East Wenatchee
Edgewood
Edmonds
Electric City
Ellensburg
Elma
Entiat
Enumclaw
Ephrata
Everett
Everson
Federal Way
Ferndale
Fife
Fircrest
Forks
George
Gig Harbor
Gold Bar
Goldendale
Grand Coulee
Grandview
Granger
Granite Falls
Harrington
Hoquiam
Ilwaco
Issaquah
Kahlotus
Kalama
Kelso
Kenmore
Kennewick
Kent
Kettle Falls
Kirkland
Kittitas
La Center
Lacey
Lake Forest Park
Lake Stevens
Lakewood
Langley
Leavenworth
Liberty Lake
Long Beach
Longview
Lynden
Lynnwood
Mabton
Maple Valley
Marysville
Mattawa
McCleary
Medical Lake
Medina
Mercer Islan

In [33]:
pnw_venues.to_csv('PNWVenues.csv', sep=',',index=False)

In [34]:
# Check the size of dataframe
print(pnw_venues.shape)
pnw_venues.head()

(5934, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aberdeen,46.97551,-123.815517,Rediviva Restaurant,46.974826,-123.817105,New American Restaurant
1,Aberdeen,46.97551,-123.815517,Sucher & Sons Star Wars Shop,46.976254,-123.813901,Toy / Game Store
2,Aberdeen,46.97551,-123.815517,Billy's Bar and Grill,46.975505,-123.813540,Bar
3,Aberdeen,46.97551,-123.815517,Amore' Italian Restaurant,46.973007,-123.817937,Italian Restaurant
4,Aberdeen,46.97551,-123.815517,Breakwater Seafood And Chowder,46.975644,-123.811814,Seafood Restaurant


In [35]:
# Tally up the total venues per city
grouped = pnw_venues.groupby('City').count()
print('Original count of cities: {}'.format(len(grouped.index)))

# Drop cities with inadequate amount of venue data (they skew the clustering results)
grouped = grouped[grouped.Venue < 10]
list(grouped.index.values)
pnw_venues = pnw_venues[~pnw_venues['City'].isin(list(grouped.index.values))]
grouped2 = pnw_venues.groupby('City').count()

print('Count of cities with more than 10 venues: {}'.format(len(grouped2.index)))

Original count of cities: 593
Count of cities with more than 10 venues: 180


In [37]:
print('There are {} unique categories.'.format(len(pnw_venues['Venue Category'].unique())))

There are 308 unique categories.


In [38]:
# Start k-means clustering
# one hot encoding
pnw_onehot = pd.get_dummies(pnw_venues[['Venue Category']], prefix="", prefix_sep="")

# Add city column back to dataframe
pnw_onehot['City'] = pnw_venues['City'] 

# move city column to the first column
fixed_columns = [pnw_onehot.columns[-1]] + list(pnw_onehot.columns[:-1])
pnw_onehot = pnw_onehot[fixed_columns]

# Check size of new dataframe
pnw_onehot.shape

(4272, 309)

In [39]:
pnw_grouped = pnw_onehot.groupby('City').mean().reset_index()
pnw_grouped

,City,ATM,Accessories Store,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Inn,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,RV Park,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Sta

In [40]:
# New dataframe size
pnw_grouped.shape

(180, 309)

Now we can find the five most common venues for each city.

In [41]:
num_top_venues = 5

for city in pnw_grouped['City']:
    print("----"+city+"----")
    temp = pnw_grouped[pnw_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aberdeen----
                  venue  freq
0         Grocery Store  0.08
1           Coffee Shop  0.08
2                  Bank  0.08
3  Fast Food Restaurant  0.08
4     Convenience Store  0.08


----Airway Heights----
                 venue  freq
0       Sandwich Place  0.09
1                  Bar  0.09
2        Grocery Store  0.09
3  Japanese Restaurant  0.09
4                 Café  0.09


----Astoria ----
                 venue  freq
0              Brewery  0.11
1   Seafood Restaurant  0.09
2                  Bar  0.07
3  American Restaurant  0.07
4          Pizza Place  0.05


----Auburn----
                 venue  freq
0          Coffee Shop  0.12
1        Grocery Store  0.08
2  American Restaurant  0.08
3                 Café  0.08
4           Restaurant  0.04


----Baker City ----
                 venue  freq
0  American Restaurant  0.18
1   Chinese Restaurant  0.18
2            Multiplex  0.09
3                 Café  0.09
4     Motorsports Shop  0.09


----Bandon----
       

In [42]:
# Function to sort venues in decscending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
# Create a dataframe with top 5 venues for each city
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = northwest_grouped['City']

for ind in np.arange(pnw_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pnw_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,Fast Food Restaurant,Bank,Grocery Store,Coffee Shop,Convenience Store
1,Airway Heights,Grocery Store,Café,Chinese Restaurant,Park,Korean Restaurant
2,Astoria,Brewery,Seafood Restaurant,American Restaurant,Bar,Pizza Place
3,Auburn,Coffee Shop,Grocery Store,American Restaurant,Café,Bank
4,Baker City,American Restaurant,Chinese Restaurant,Motorsports Shop,Bagel Shop,Automotive Shop


In [91]:
# Run K-means to break up into clusters
kclusters = 6

pnw_grouped_clustering = pnw_grouped.drop('City', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pnw_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:400]

array([4, 2, 1, 4, 1, 1, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 2, 4, 2, 2, 4, 4,
       1, 1, 2, 2, 2, 2, 2, 4, 2, 1, 4, 4, 2, 3, 4, 2, 1, 1, 2, 4, 4, 4,
       4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 4, 3, 1, 4, 2, 2, 4, 2, 1, 2, 4,
       2, 4, 1, 2, 2, 4, 1, 3, 3, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 2, 4, 1,
       4, 1, 3, 0, 2, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 1, 2, 4,
       4, 3, 4, 4, 4, 4, 3, 2, 3, 1, 4, 4, 1, 4, 3, 5, 4, 1, 2, 4, 4, 4,
       2, 3, 1, 4, 2, 2, 4, 4, 2, 4, 4, 3, 4, 5, 4, 4, 4, 1, 4, 2, 4, 4,
       4, 2, 2, 1, 4, 1, 4, 4, 4, 4, 3, 4, 4, 4, 4, 1, 4, 4, 2, 1, 2, 2,
       2, 2, 4, 4])

In [92]:
cities_venues_sorted.columns

Index(['City', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue'],
      dtype='object')

In [93]:
# Create dataframe that includes the cluster and top 5 venues

# Add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pnw_merged = df_northwest

# Merge northwest_grouped with northwest_data to add latitude/longitude for each city
pnw_merged = pnw_merged.join(cities_venues_sorted.set_index('City'), on='City')

# Drop cities with no venue data
pnw_merged = pnw_merged.dropna()

pnw_merged

,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,WA,46.9755,-123.816,4.0,Fast Food Restaurant,Bank,Grocery Store,Coffee Shop,Convenience Store
1,Airway Heights,WA,47.643,-117.593,2.0,Grocery Store,Café,Chinese Restaurant,Park,Korean Restaurant
6,Auburn,WA,47.3075,-122.226,4.0,Coffee Shop,Grocery Store,American Restaurant,Café,Bank
8,Battle Ground,WA,45.7807,-122.548,4.0,Coffee Shop,Pizza Place,Hotel,Asian Restaurant,Discount Store
9,Bellevue,WA,47.6137,-122.191,4.0,Coffee Shop,Hotel,Mexican Restaurant,Women's Store,Bakery
10,Bellingham,WA,48.7549,-122.478,4.0,Coffee Shop,Café,Bookstore,Chinese Restaurant,Breakfast Spot
12,Bingen,WA,45.7148,-121.466,2.0,Mexican Restaurant,Train Station,Food Truck,Park,Pizza Place
14,Blaine,WA,48.994,-122.752,4.0,Coffee Shop,Diner,ATM,Steakhouse,Taco Place
16,Bothell,WA,47.7601,-122.205,4.0,Coffee Shop,Mexican Restaurant,Pub,Park,Sushi Restaurant
17,Bremerton,WA,47.5674,-122.633,4.0,Coffee Shop,Asian Restaurant,Cocktail Bar,Arcade,Mexican Restaurant


In [94]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pnw_merged['Latitude'], pnw_merged['Longitude'], pnw_merged['City'], pnw_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0: Outlier city
This cluster consists of a single outlier city, Lynden WA. Although none of its top common venues seems uncommon in
itself, perhaps it was the combination of all 5 that proved particularly unique among our dataset.

In [132]:
pnw_merged.loc[pnw_merged['Cluster Labels'] == 0, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
95,Lynden,WA,Bakery,Sandwich Place,Playground,Coffee Shop,Café


#### Cluster 1: Vacation destinations
Cluster 1 is characterized by hotels/resorts, restaurants and nightlife (bars, breweries, etc). Many of the cities on this
list are vacation destinations and/or popular weekend getaway spots.

In [139]:
# Cluster 1
pnw_merged.loc[pnw_merged['Cluster Labels'] == 1, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
38,Colville,WA,American Restaurant,Mexican Restaurant,Other Repair Shop,Automotive Shop,Coffee Shop
43,Dayton,WA,American Restaurant,Hotel,Restaurant,Bakery,Asian Restaurant
74,Ilwaco,WA,Harbor / Marina,Seafood Restaurant,Café,Grocery Store,High School
91,Leavenworth,WA,Park,Hotel,Resort,Candy Store,American Restaurant
114,Mount Vernon,WA,American Restaurant,Bar,Thai Restaurant,Italian Restaurant,Indian Restaurant
141,Port Townsend,WA,Hotel,American Restaurant,Thai Restaurant,Café,Seafood Restaurant
146,Puyallup,WA,Hotel,Deli / Bodega,American Restaurant,Event Space,Coffee Shop
178,Stevenson,WA,American Restaurant,Restaurant,Coffee Shop,Seafood Restaurant,Food & Drink Shop
204,White Salmon,WA,Grocery Store,Café,Coffee Shop,Paper / Office Supplies Store,Bakery
244,Pendleton,OR,Hotel,Steakhouse,Café,Restaurant,Convenience Store


#### Cluster 2: : Restaurant cities
Pizza place is common among nearly all of the cities in this cluster. It looks like Pizza Place, Mexican Restaurant, Chinese Restaurant, American Restaurant, and Bar are all grouping together here. The cities on this list tend to be larger than bedroom communities, but somewhat smaller than major uban centers.

In [138]:
# Cluster 2
pnw_merged.loc[pnw_merged['Cluster Labels'] == 2, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Airway Heights,WA,Grocery Store,Café,Chinese Restaurant,Park,Korean Restaurant
12,Bingen,WA,Mexican Restaurant,Train Station,Food Truck,Park,Pizza Place
23,Burlington,WA,Pizza Place,Pub,Farmers Market,Café,Convenience Store
25,Carnation,WA,Pizza Place,Café,Supermarket,Vietnamese Restaurant,Bar
33,Clarkston,WA,Restaurant,Pharmacy,Thai Restaurant,Mobile Phone Shop,Pizza Place
52,Ellensburg,WA,Brewery,American Restaurant,Bookstore,Bar,Grocery Store
55,Enumclaw,WA,Pizza Place,Pub,Bakery,Supermarket,Breakfast Spot
56,Ephrata,WA,Pharmacy,ATM,American Restaurant,Asian Restaurant,Park
60,Ferndale,WA,Bar,Indian Restaurant,Convenience Store,Fast Food Restaurant,Park
71,Granite Falls,WA,Asian Restaurant,Trail,Pizza Place,Mexican Restaurant,Steakhouse


#### Cluster 3: Fast food cities

The unifying characteristic among these cities is that *Fast Food Restaurant* is the prominent venue type. These are smaller cities and bedroom communities that tend to be located between larger cities with more ammentiies. At first glance, a "Fast food city" might not seem particularly attractive, but this cluster
deserves further exploration for prospective home buyers looking for more seclusion and lower real estate prices.

In [135]:
# Cluster 3 
pnw_merged.loc[pnw_merged['Cluster Labels'] == 3, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,Covington,WA,Fast Food Restaurant,Sandwich Place,Automotive Shop,Big Box Store,Pharmacy
94,Longview,WA,Fast Food Restaurant,ATM,Pizza Place,Coffee Shop,Asian Restaurant
98,Maple Valley,WA,Fast Food Restaurant,Burger Joint,Mexican Restaurant,Seafood Restaurant,Gas Station
99,Marysville,WA,Fast Food Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Supermarket
136,Pasco,WA,Fast Food Restaurant,Pizza Place,Pharmacy,Mexican Restaurant,Grocery Store
153,Richland,WA,Fast Food Restaurant,Sandwich Place,Garden Center,Hotel,Taco Place
224,Keizer,OR,Hotel,Pharmacy,Mexican Restaurant,Fast Food Restaurant,Optical Shop
226,Oregon City,OR,Fast Food Restaurant,Coffee Shop,Sandwich Place,ATM,Pharmacy
263,Prineville,OR,Bank,Sandwich Place,Burger Joint,Salad Place,Hotel
278,Seaside,OR,Hotel,Fast Food Restaurant,Candy Store,Breakfast Spot,Pizza Place


#### Cluster 4: Coffee shop cities
In addition to coffee shops being the prevalent venue type, the cities in this cluster are charaterized
by a diverse set of ammenities, indicative of larger urban centers.

In [136]:
# Cluster 4
pnw_merged.loc[pnw_merged['Cluster Labels'] == 4, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,WA,Fast Food Restaurant,Bank,Grocery Store,Coffee Shop,Convenience Store
6,Auburn,WA,Coffee Shop,Grocery Store,American Restaurant,Café,Bank
8,Battle Ground,WA,Coffee Shop,Pizza Place,Hotel,Asian Restaurant,Discount Store
9,Bellevue,WA,Coffee Shop,Hotel,Mexican Restaurant,Women's Store,Bakery
10,Bellingham,WA,Coffee Shop,Café,Bookstore,Chinese Restaurant,Breakfast Spot
14,Blaine,WA,Coffee Shop,Diner,ATM,Steakhouse,Taco Place
16,Bothell,WA,Coffee Shop,Mexican Restaurant,Pub,Park,Sushi Restaurant
17,Bremerton,WA,Coffee Shop,Asian Restaurant,Cocktail Bar,Arcade,Mexican Restaurant
22,Burien,WA,Mexican Restaurant,Bakery,Ice Cream Shop,Italian Restaurant,New American Restaurant
24,Camas,WA,Coffee Shop,Mexican Restaurant,Brewery,ATM,Grocery Store


#### Cluster 5: More outliers

The yield for the final cluster was a couple more outlier cities, also in Washington state. These cities both have a
category in their top 5 venue types that is unique among our dataset: the *Gastropub* in Prosser, and the *Bowling Alley*
in Sedro-Woolley.

In [137]:
# Cluster 5
pnw_merged.loc[pnw_merged['Cluster Labels'] == 5, pnw_merged.columns[[0] + [1] + list(range(5, pnw_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
144,Prosser,WA,Brewery,American Restaurant,Coffee Shop,Pizza Place,Gastropub
164,Sedro-Woolley,WA,American Restaurant,Pizza Place,Bowling Alley,Lawyer,Convenience Store
